In [5]:
from __future__ import print_function

import sys

sys.path.insert(0, '.')

import torch
from torch.autograd import Variable
import torch.optim as optim
from torch.nn.parallel import DataParallel

import time
import os.path as osp
from tensorboardX import SummaryWriter
import numpy as np
import argparse

from tri_loss.dataset import create_dataset
from tri_loss.model.Model import Model
from tri_loss.model.TripletLoss import TripletLoss
from tri_loss.model.loss import global_loss

from tri_loss.utils.utils import time_str
from tri_loss.utils.utils import str2bool
from tri_loss.utils.utils import tight_float_str as tfs
from tri_loss.utils.utils import may_set_mode
from tri_loss.utils.utils import load_state_dict
from tri_loss.utils.utils import load_ckpt
from tri_loss.utils.utils import save_ckpt
from tri_loss.utils.utils import set_devices
from tri_loss.utils.utils import AverageMeter
from tri_loss.utils.utils import to_scalar
from tri_loss.utils.utils import ReDirectSTD
from tri_loss.utils.utils import set_seed
from tri_loss.utils.utils import adjust_lr_exp
from tri_loss.utils.utils import adjust_lr_staircase



/home/bmsknight/anaconda3/envs/pytorch/lib/python2.7/site-packages/scikit_learn-0.20.0-py2.7-linux-x86_64.egg/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [4]:
import sys
import os.path
path = os.path.dirname(os.path.dirname(os.path.abspath("/home/bmsknight/triplet/person-reid-triplet-loss-baseline/tri_loss/dataset/")))
sys.path.insert(0,path)


In [12]:

class ExtractFeature(object):
    """A function to be called in the val/test set, to extract features.
    Args:
    TVT: A callable to transfer images to specific device.
    """

    def __init__(self, model, TVT):
        self.model = model
        self.TVT = TVT

    def extract(self, ims):
        old_train_eval_model = self.model.training
        # Set eval mode.
        # Force all BN layers to use global mean and variance, also disable
        # dropout.
        self.model.eval()
        ims = Variable(self.TVT(torch.from_numpy(ims).float()))
        feat = self.model(ims)
        feat = feat.data.cpu().numpy()
        # Restore the model to its old train/eval mode.
        self.model.train(old_train_eval_model)
        return feat


In [9]:
def test(load_model_weight=True):
    if load_model_weight:
      if cfg.model_weight_file != '':
        map_location = (lambda storage, loc: storage)
        sd = torch.load(cfg.model_weight_file, map_location=map_location)
        load_state_dict(model, sd)
        print('Loaded model weights from {}'.format(cfg.model_weight_file))
      else:
        load_ckpt(modules_optims, cfg.ckpt_file)

    for test_set, name in zip(test_sets, test_set_names):
      test_set.set_feat_func(ExtractFeature(model_w, TVT))
      print('\n=========> Test on dataset: {} <=========\n'.format(name))
      test_set.eval(
        normalize_feat=cfg.normalize_feature,
        verbose=True)

In [8]:
TVT, TMO = set_devices((0,))

In [6]:
class Config(object):
    def __init__(self):
        self.resize_h_w = (256,128)
        self.scale_im = True
        self.im_mean = [0.486, 0.459, 0.408]
        self.im_std = [0.229, 0.224, 0.225]
        self.device_id = (0,)
        self.model_weight_file = '/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/market/model_weight.pth'
        self.margin = 0.3
        
        self.last_conv_stride = 1
        self.weight_decay = 0.0005

        # Initial learning rate
        self.base_lr = 0.0002
        

In [7]:
cfg = Config()

In [14]:
model = Model(last_conv_stride=cfg.last_conv_stride)
  # Model wrapper
model_w = DataParallel(model)

#############################
# Criteria and Optimizers   #
#############################

tri_loss = TripletLoss(margin=cfg.margin)

optimizer = optim.Adam(model.parameters(),
                     lr=cfg.base_lr,
                     weight_decay=cfg.weight_decay)

# Bind them together just to save some codes in the following usage.
modules_optims = [model, optimizer]


In [15]:
map_location = (lambda storage, loc: storage)
sd = torch.load(cfg.model_weight_file, map_location=map_location)
load_state_dict(model, sd)
print('Loaded model weights from {}'.format(cfg.model_weight_file))

Keys not found in source state_dict: 
	 base.layer2.0.bn1.num_batches_tracked
	 base.layer4.1.bn3.num_batches_tracked
	 base.layer4.0.bn1.num_batches_tracked
	 base.layer3.5.bn1.num_batches_tracked
	 base.layer3.2.bn2.num_batches_tracked
	 base.layer1.0.bn1.num_batches_tracked
	 base.layer1.1.bn2.num_batches_tracked
	 base.bn1.num_batches_tracked
	 base.layer3.1.bn3.num_batches_tracked
	 base.layer2.0.bn3.num_batches_tracked
	 base.layer2.0.downsample.1.num_batches_tracked
	 base.layer4.0.bn2.num_batches_tracked
	 base.layer2.2.bn3.num_batches_tracked
	 base.layer3.4.bn2.num_batches_tracked
	 base.layer3.4.bn3.num_batches_tracked
	 base.layer1.0.bn3.num_batches_tracked
	 base.layer1.1.bn1.num_batches_tracked
	 base.layer4.0.bn3.num_batches_tracked
	 base.layer1.1.bn3.num_batches_tracked
	 base.layer3.0.bn3.num_batches_tracked
	 base.layer2.3.bn1.num_batches_tracked
	 base.layer3.2.bn3.num_batches_tracked
	 base.layer3.0.bn2.num_batches_tracked
	 base.layer4.2.bn3.num_batches_tracked
	 

In [107]:
def normalize(nparray, order=2, axis=0):
    """Normalize a N-D numpy array along the specified axis."""
    norm = np.linalg.norm(nparray, ord=order, axis=axis, keepdims=True)
    return nparray / (norm + np.finfo(np.float32).eps)


def compute_dist(array1, array2, type='euclidean'):

    if type == 'cosine':
        array1 = normalize(array1, axis=1)
        array2 = normalize(array2, axis=1)
        dist = np.matmul(array1, array2.T)
        return dist
    else:
        # shape [m1, 1]
        square1 = np.sum(np.square(array1), axis=1)[..., np.newaxis]
        # shape [1, m2]
        square2 = np.sum(np.square(array2), axis=1)[np.newaxis, ...]
        squared_dist = - 2 * np.matmul(array1, array2.T) + square1 + square2
        squared_dist[squared_dist < 0] = 0
        dist = np.sqrt(squared_dist)
        return dist


In [17]:
def pre_process_im(cfg, im):
    """Pre-process image.
    `im` is a numpy array with shape [H, W, 3], e.g. the result of
    matplotlib.pyplot.imread(some_im_path), or
    numpy.asarray(PIL.Image.open(some_im_path))."""

    
    # Resize.
    if (cfg.resize_h_w is not None) \
        and (cfg.resize_h_w != (im.shape[0], im.shape[1])):
      im = cv2.resize(im, cfg.resize_h_w[::-1], interpolation=cv2.INTER_LINEAR)

    # scaled by 1/255.
    if cfg.scale_im:
      im = im / 255.

    # Subtract mean and scaled by std
    # im -= np.array(self.im_mean) # This causes an error:
    # Cannot cast ufunc subtract output from dtype('float64') to
    # dtype('uint8') with casting rule 'same_kind'
    if cfg.im_mean is not None:
      im = im - np.array(cfg.im_mean)
    if cfg.im_mean is not None and cfg.im_std is not None:
      im = im / np.array(cfg.im_std).astype(float)

    # May mirror image.
    

    # The original image has dims 'HWC', transform it to 'CHW'.
    im = im.transpose(2, 0, 1)

    return im

In [60]:

#im1 = im1.transpose(2,0,1)

In [18]:
import PIL
import cv2

In [19]:
ext = ExtractFeature(model=model_w,TVT=TVT)


In [66]:
im1 = np.asarray(PIL.Image.open("/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/query/priyan5.jpg"))
im2 = np.asarray(PIL.Image.open("/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/Dynamic_Database/piriyanthan/piriyanthan_26_3.jpg"))

In [67]:
im1 = pre_process_im(cfg, im1)
im2 = pre_process_im(cfg, im2)
im1 = np.reshape(im1,(1,3,256,128))
im2 = np.reshape(im2,(1,3,256,128))
ft1 = ext.extract(im1)
ft2 = ext.extract(im2)
val = compute_dist(ft1,ft2)

In [68]:
val

array([[ 1.61921597]], dtype=float32)

In [142]:
root_directory = '/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/Dynamic_Database/'
folder_list = os.listdir(root_directory)


global_feature_list = None
name_list = []
for name in folder_list:
    glist = []
    
    directory = root_directory + name + '/'

    for filename in os.listdir(directory):
        input_image = np.asarray(PIL.Image.open(directory+filename))
        input_image = pre_process_im(cfg, input_image)
        input_image = np.reshape(input_image,(1,3,256,128))
        feature = ext.extract(input_image)
        glist.append(feature)
    glist = np.asarray(glist).reshape(-1,2048)
    if global_feature_list ==None:
        global_feature_list = glist
    else:
        global_feature_list= np.vstack((global_feature_list, glist))
    print ([name] * glist.shape[0])
    name_list = name_list + ([name] * glist.shape[0])

['piriyanthan', 'piriyanthan', 'piriyanthan', 'piriyanthan', 'piriyanthan', 'piriyanthan']
['thivakaran', 'thivakaran', 'thivakaran', 'thivakaran', 'thivakaran']


/home/bmsknight/anaconda3/envs/pytorch/lib/python2.7/site-packages/ipykernel_launcher.py:19: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


['madhushan', 'madhushan', 'madhushan', 'madhushan', 'madhushan', 'madhushan', 'madhushan', 'madhushan', 'madhushan', 'madhushan']
['harishanth', 'harishanth', 'harishanth', 'harishanth', 'harishanth']
['athavan', 'athavan', 'athavan', 'athavan', 'athavan', 'athavan', 'athavan']


In [ ]:
glo

In [169]:
querry_image = np.asarray(PIL.Image.open("/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/query/person1_frameNo_20.jpg"))
querry_image = pre_process_im(cfg, querry_image)
querry_image = np.reshape(querry_image,(1,3,256,128))
querry_feature = ext.extract(querry_image)
min_dist_list = {}

dist = compute_dist(global_feature_list, querry_feature, type='cosine')

In [170]:
name_list[np.argmax(dist)]

'thivakaran'

[array([[ 0.06824015,  0.20392846,  0.1027478 , ...,  0.04803802,
          0.11008131,  0.08103167]], dtype=float32),
 array([[ 0.06824015,  0.20392846,  0.1027478 , ...,  0.04803802,
          0.11008131,  0.08103167]], dtype=float32),
 array([[ 0.06824015,  0.20392846,  0.1027478 , ...,  0.04803802,
          0.11008131,  0.08103167]], dtype=float32),
 array([[ 0.06824015,  0.20392846,  0.1027478 , ...,  0.04803802,
          0.11008131,  0.08103167]], dtype=float32),
 array([[ 0.06824015,  0.20392846,  0.1027478 , ...,  0.04803802,
          0.11008131,  0.08103167]], dtype=float32)]

In [171]:
querrypath = "/home/bmsknight/triplet/person-reid-triplet-loss-baseline/data/ourdata/Database/query/"
for filename in os.listdir(querrypath):
    querry_image = np.asarray(PIL.Image.open(querrypath+filename))
    querry_image = pre_process_im(cfg, querry_image)
    querry_image = np.reshape(querry_image,(1,3,256,128))
    querry_feature = ext.extract(querry_image)
    
    cos_dist = compute_dist(global_feature_list, querry_feature, type='cosine')
    cos_pred = name_list[np.argmax(cos_dist)]
    
    euc_dist = compute_dist(global_feature_list, querry_feature, type='euclidean')
    euc_pred = name_list[np.argmin(euc_dist)]
    
    print(filename, cos_pred, euc_pred)
    

priyan7.jpg piriyanthan piriyanthan
person3_frameNo_240.jpg athavan athavan
priyan16.jpg madhushan madhushan
person0_frameNo_240.jpg harishanth harishanth
person0_frameNo_220.jpg harishanth harishanth
person0_frameNo_180.jpg harishanth harishanth
person0_frameNo_260.jpg harishanth harishanth
person0_frameNo_80.jpg harishanth harishanth
person3_frameNo_340.jpg athavan athavan
person2_frameNo_40.jpg thivakaran thivakaran
person4_frameNo_160.jpg thivakaran thivakaran
person2_frameNo_280.jpg thivakaran thivakaran
person2_frameNo_240.jpg thivakaran thivakaran
person0_frameNo_160.jpg harishanth harishanth
person4_frameNo_20.jpg thivakaran thivakaran
person4_frameNo_140.jpg thivakaran thivakaran
person4_frameNo_280.jpg athavan athavan
person4_frameNo_200.jpg thivakaran thivakaran
person4_frameNo_60.jpg thivakaran thivakaran
person2_frameNo_140.jpg thivakaran thivakaran
person3_frameNo_320.jpg athavan athavan
person2_frameNo_340.jpg thivakaran thivakaran
person0_frameNo_380.jpg harishanth hari